In [40]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [41]:
%cd /content/drive/MyDrive/Colab/IoTDevice/Federated_IID_Best/FocalLoss


/content/drive/MyDrive/Colab/IoTDevice/Federated_IID_Best/FocalLoss


In [42]:
!ls

flexperimentIIDfocalloss.ipynb	gamma5alphanone


In [43]:
import torch
import torch.nn as nn
from sklearn.metrics import f1_score, precision_score, recall_score

import numpy as np
import pandas as pd
import random
import cv2
import os

from imutils import paths
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score, precision_recall_curve, precision_score, recall_score, average_precision_score

# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Flatten, Dense, Dropout
# from tensorflow.keras.optimizers import SGD, Adam
# from tensorflow.keras import backend as K


import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


In [44]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import os

folder_path = '/content/drive/MyDrive/Colab/IoTDevice/Feature'
file_names = [file for file in os.listdir(folder_path) if file.endswith('.csv')]
# print(file_names)

# mapping_df = pd.DataFrame({'csv_file': file_names, 'device': range(len(file_names))})
# mapping_df.to_csv('/content/drive/MyDrive/Colab/IoTDevice/device_map.csv', index=False)

dfs = []
for i, file_name in enumerate(file_names):
    file_path = os.path.join(folder_path, file_name)
    df = pd.read_csv(file_path)
    df['device'] = i
    dfs.append(df)

# Concatenate all DataFrames into a single DataFrame
df = pd.concat(dfs, ignore_index=True)


In [45]:
# df = pd.concat([df1, df2], axis=0)
num_classes = len(df['device'].unique())

In [46]:
num_classes

18

In [47]:
def find_best_n_components(df, input_len, max_components=None, min_variance_ratio=0.95):
    x = df.iloc[:, 0:input_len]
    y = df.iloc[:, input_len:(input_len + 1)]
    if max_components is None:
        max_components = min(x.shape)

    pca = PCA(n_components=max_components)
    pca.fit(x)
    explained_variance_ratio = pca.explained_variance_ratio_
    cumulative_variance_ratio = np.cumsum(explained_variance_ratio)

    best_n_components = np.argmax(cumulative_variance_ratio >= min_variance_ratio) + 1
    # Ensure the number of components is divisible by 4
    while best_n_components % 8 != 0:
        best_n_components -= 1

    return best_n_components

In [48]:
n_components = find_best_n_components(df, 100, None, 0.95)

In [49]:
n_components

80

In [50]:
x = df.iloc[:, 0:100].values
y = df.iloc[:,100:101].values

# # Apply PCA for dimensionality reduction
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(x)  # Standardize features
# pca = PCA(n_components=n_components)  # Choose the desired number of components
# X_pca = pca.fit_transform(X_scaled)  # Apply PCA

In [51]:
type(x)

numpy.ndarray

In [52]:
# Apply PCA for dimensionality reduction
scaler = StandardScaler()
X_scaled = scaler.fit_transform(x)  # Standardize features
pca = PCA(n_components=n_components)  # Choose the desired number of components
X_pca = pca.fit_transform(X_scaled)

In [53]:
# Define the number of clients
num_clients = 5  # Adjust as needed
# Define batch size for DataLoader
batch_size = 100  # Batch size for each clients

**Create Clients and Divide X_pca and y among num_clients**

In [74]:
import torch
from torch.utils.data import TensorDataset, DataLoader
import pandas as pd
import numpy as np

# Assuming you have x and y DataFrames
# Convert them to PyTorch tensors
x_tensor = torch.tensor(X_pca, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.long)  # Assuming y contains integer labels

# Create a TensorDataset
dataset = TensorDataset(x_tensor, y_tensor)



# Create DataLoader objects for training and validation sets
# Here, we use 80% of the data for training and 20% for validation, but you can adjust the split as needed
train_size = int(0.8 * len(dataset))
test_size = int(0.1 * len(dataset))
val_size = len(dataset) - train_size - test_size
train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, val_size, test_size])


client_train_datasets = []
start_index = 0
for i in range(num_clients):
    subset_size = train_size // num_clients + (1 if i < train_size % num_clients else 0)
    subset = torch.utils.data.Subset(train_dataset, range(start_index, start_index + subset_size))
    client_train_datasets.append(subset)
    start_index += subset_size

# Create DataLoader objects for training and validation sets for each client
train_loaders = []
val_loaders = []
test_loaders = []
for client_train_dataset in client_train_datasets:
    client_train_loader = DataLoader(client_train_dataset, batch_size=batch_size, shuffle=True)
    train_loaders.append(client_train_loader)
    val_loaders.append(DataLoader(val_dataset, batch_size=batch_size))
    test_loaders.append(DataLoader(test_dataset, batch_size=batch_size))



In [ ]:
# import torch
# from torch.utils.data import TensorDataset, DataLoader
# import pandas as pd
# import numpy as np

# # Assuming you have x and y DataFrames
# # Convert them to PyTorch tensors
# x_tensor = torch.tensor(X_pca, dtype=torch.float32)
# y_tensor = torch.tensor(y, dtype=torch.long)  # Assuming y contains integer labels

# # Create a TensorDataset
# dataset = TensorDataset(x_tensor, y_tensor)



# # Create DataLoader objects for training and validation sets
# # Here, we use 80% of the data for training and 20% for validation, but you can adjust the split as needed
# train_size = int(0.8 * len(dataset))
# test_size = int(0.1 * len(dataset))
# val_size = len(dataset) - train_size - test_size
# train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])


# client_train_datasets = []
# start_index = 0
# for i in range(num_clients):
#     subset_size = train_size // num_clients + (1 if i < train_size % num_clients else 0)
#     subset = torch.utils.data.Subset(train_dataset, range(start_index, start_index + subset_size))
#     client_train_datasets.append(subset)
#     start_index += subset_size

# # Create DataLoader objects for training and validation sets for each client
# train_loaders = []
# val_loaders = []
# for client_train_dataset in client_train_datasets:
#     client_train_loader = DataLoader(client_train_dataset, batch_size=batch_size, shuffle=True)
#     train_loaders.append(client_train_loader)
#     val_loaders.append(DataLoader(val_dataset, batch_size=batch_size))


**Define Transformer Model**

In [75]:
class TransformerClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, num_classes):
        super(TransformerClassifier, self).__init__()
        self.hidden_dim = hidden_dim
        self.encoder_layers = TransformerEncoderLayer(input_dim, nhead=8, dim_feedforward=hidden_dim)
        self.transformer_encoder = TransformerEncoder(self.encoder_layers, num_layers)
        self.fc1 = nn.Linear(hidden_dim, 32)
        self.fc2 = nn.Linear(32, num_classes)


    def forward(self, x):
        # x: (seq_len, batch_size, input_dim)
        x = self.transformer_encoder(x)
        # take the output from the last time step
        x = x[-1, :, :]
        x = self.fc1(x)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)  # Log softmax for numerical stability

**Initialize Client Models**

In [56]:
input_dim = n_components  # Dimension of input embeddings (output of PCA)
hidden_dim = n_components  # Hidden dimension of the Transformer
num_layers = 4  # Number of Transformer layers
num_classes = 18  # Number of output classes

global_model = TransformerClassifier(input_dim, hidden_dim, num_layers, num_classes)
client_models = [global_model for _ in range(num_clients)]

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [57]:
# import torch
# criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# # Create a tensor of zeros with shape (1, 100, 80)
# data = torch.zeros(1, 100, n_components)
# label = torch.randint(0, 2, (100,), dtype=torch.long)


# # Pass the input tensor through the model
# out = global_model(data)
# print(out.shape)
# print(label.shape)
# loss = criterion(out, label)
# print(loss)

**Now we have train_loader for each client and model for each client, we proceed for federated training**

In [58]:
# Aggregate local models to a global model after each round of local training
def aggregate_model_updates(global_model, client_models):
    global_state_dict = global_model.state_dict()
    for key in global_state_dict.keys():
        # Aggregate the parameter values from all client models
        aggregated_param = torch.stack([client.state_dict()[key] for client in client_models], dim=0).mean(dim=0)
        global_state_dict[key] = aggregated_param
    global_model.load_state_dict(global_state_dict)
    return global_model

In [59]:
# def train_federated(global_model, client_models, train_loaders, val_loaders, num_epochs, criterion, optimizer, device):

#   best_val_loss = float('inf')
#   best_model_path = '/content/drive/MyDrive/Colab/IoTDevice/Federated_IID_Best/best_model_focalloss.pth'
#   epoch_list = []
#   accuracy_list = []
#   val_loss_list = []
#   train_loss_list = []  # Added to track training loss
#   f1_list = []
#   precision_list = []
#   recall_list = []
#   f1_class_lists = []

#   # Load the best model if it exists
#   if os.path.exists(best_model_path):
#       global_model.load_state_dict(torch.load(best_model_path))
#       print("Loaded previously trained model")

#   for epoch in range(num_epochs):
#       client_model_losses = []
#       for client_model, train_loader in zip(client_models, train_loaders):
#         client_model.train()
#         running_train_loss = 0.0
#         for inputs, labels in train_loader:
#           inputs, labels = inputs.to(device), labels.to(device)
#           inputs = torch.unsqueeze(inputs, 0)
#           labels = torch.squeeze(labels, dim=1)
#           optimizer.zero_grad()
#           outputs = client_model(inputs)
#           loss =  criterion(outputs, labels)
#           loss.backward()
#           optimizer.step()
#           running_train_loss += loss.item() * inputs.size(0)
#         client_model_losses.append(running_train_loss / len(train_loader.dataset))

#       global_model = aggregate_model_updates(global_model, client_models)
#       # Compute average training loss for all client models
#       print(client_model_losses)
#       epoch_train_loss = sum(client_model_losses) / len(client_models)
#       train_loss_list.append(epoch_train_loss)  # Append training loss to list

#       # Validation loop
#       global_model.eval()  # Set the global model to evaluation mode
#       val_loss = 0.0
#       correct = 0
#       total = 0
#       all_predicted = []
#       all_labels = []
#       with torch.no_grad():
#           for val_loader in val_loaders:
#               for inputs, labels in val_loader:
#                   inputs, labels = inputs.to(device), labels.to(device)
#                   inputs = torch.unsqueeze(inputs, 0)
#                   labels = torch.squeeze(labels, dim=1)
#                   outputs = global_model(inputs)  # Forward pass using global model
#                   val_loss += criterion(outputs, labels).item()  # Compute the loss
#                   _, predicted = torch.max(outputs, 1)
#                   all_predicted.extend(predicted.tolist())
#                   all_labels.extend(labels.tolist())
#                   total += labels.size(0)
#                   correct += (predicted == labels).sum().item()

#       # Calculate metrics
#       accuracy = 100 * correct / total
#       val_loss /= sum(len(val_loader.dataset) for val_loader in val_loaders)
#       f1 = f1_score(all_labels, all_predicted, average='macro')
#       precision = precision_score(all_labels, all_predicted, average='macro')
#       recall = recall_score(all_labels, all_predicted, average='macro')
#       f1_class = f1_score(all_labels, all_predicted, average=None)

#       # Append results to lists
#       epoch_list.append(epoch + 601)
#       accuracy_list.append(accuracy)
#       val_loss_list.append(val_loss)
#       f1_list.append(f1)
#       precision_list.append(precision)
#       recall_list.append(recall)
#       f1_class_lists.append(f1_class)

#       # Save the model after each epoch if validation loss improves
#       if val_loss < best_val_loss:
#           torch.save(global_model.state_dict(), best_model_path)
#           best_val_loss = val_loss

#       # Print epoch statistics
#       print(f'Epoch {epoch+1}/{num_epochs}, Training Loss: {epoch_train_loss:.4f}, Validation Loss: {val_loss:.4f}, Accuracy: {accuracy:.2f}%, F1 Score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}')
#       for i in range(len(f1_class)):
#           print(f'Class {i}: F1 Score: {f1_class[i]:.4f}')

#   # Create result DataFrame
#   result_df = pd.DataFrame({
#     'Epoch': epoch_list,
#     'Accuracy': accuracy_list,
#     'Training Loss': train_loss_list,  # Include training loss in DataFrame
#     'Validation Loss': val_loss_list,
#     'F1 Score': f1_list,
#     'Precision': precision_list,
#     'Recall': recall_list,
#     'f1_score': f1_class_lists
#   })

#   # Save result_df to a CSV file
#   result_df.to_csv('result_df_fl_iid_focalloss.csv', index=False)

#   # return global_model


In [60]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import f1_score, precision_score, recall_score
import pandas as pd
import os

def train_federated(global_model, client_models, train_loaders, val_loaders, num_epochs, criterion, optimizer, device):

    best_val_loss = float('inf')
    best_model_path = '/content/drive/MyDrive/Colab/IoTDevice/Federated_IID_Best/FocalLoss/best_model_focalloss.pth'
    epoch_list = []
    train_loss_list = []  # Added to track training loss
    val_loss_list = []  # Added to track validation loss
    train_accuracy_list = []  # Added to track training accuracy
    val_accuracy_list = []  # Added to track validation accuracy
    f1_list = []
    precision_list = []
    recall_list = []
    f1_class_lists = []
    precision_class_lists = []  # Added to track class-wise precision
    recall_class_lists = []     # Added to track class-wise recall

    # Load the best model if it exists
    if os.path.exists(best_model_path):
        global_model.load_state_dict(torch.load(best_model_path))
        print("Loaded previously trained model")

    for epoch in range(num_epochs):
        client_model_losses = []
        client_model_accuracy = []
        for client_model, train_loader in zip(client_models, train_loaders):
            client_model.train()
            running_train_loss = 0.0
            total_correct_train = 0
            total_train = 0
            for inputs, labels in train_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                inputs = torch.unsqueeze(inputs, 0)
                labels = torch.squeeze(labels, dim=1)
                optimizer.zero_grad()
                outputs = client_model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                running_train_loss += loss.item() * inputs.size(0)
                _, predicted_train = torch.max(outputs, 1)
                total_train += labels.size(0)
                total_correct_train += (predicted_train == labels).sum().item()
            client_model_losses.append(running_train_loss / len(train_loader.dataset))
            epoch_train_accuracy = 100 * total_correct_train / total_train
            client_model_accuracy.append(epoch_train_accuracy)

        global_model = aggregate_model_updates(global_model, client_models)
        # Compute average training loss for all client models
        print(client_model_losses)
        epoch_train_loss = sum(client_model_losses) / len(client_models)
        epoch_train_accuracy = sum(client_model_accuracy) / len(client_models)
        train_loss_list.append(epoch_train_loss)  # Append training loss to list
        train_accuracy_list.append(epoch_train_accuracy)  # Append training loss to list

        # Validation loop
        global_model.eval()  # Set the global model to evaluation mode
        val_loss = 0.0
        total_correct_val = 0
        total_val = 0
        all_predicted = []
        all_labels = []
        with torch.no_grad():
            for val_loader in val_loaders:
                for inputs, labels in val_loader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    inputs = torch.unsqueeze(inputs, 0)
                    labels = torch.squeeze(labels, dim=1)
                    outputs = global_model(inputs)  # Forward pass using global model
                    val_loss += criterion(outputs, labels).item()  # Compute the loss
                    _, predicted = torch.max(outputs, 1)
                    all_predicted.extend(predicted.tolist())
                    all_labels.extend(labels.tolist())
                    total_val += labels.size(0)
                    total_correct_val += (predicted == labels).sum().item()

        # Calculate metrics
        val_accuracy = 100 * total_correct_val / total_val
        val_loss /= sum(len(val_loader.dataset) for val_loader in val_loaders)
        f1 = f1_score(all_labels, all_predicted, average='macro')
        precision = precision_score(all_labels, all_predicted, average='macro')
        recall = recall_score(all_labels, all_predicted, average='macro')
        f1_class = f1_score(all_labels, all_predicted, average=None)
        precision_class = precision_score(all_labels, all_predicted, average=None)  # Class-wise precision
        recall_class = recall_score(all_labels, all_predicted, average=None)  # Class-wise recall

        # Append results to lists
        epoch_list.append(epoch)
        val_accuracy_list.append(val_accuracy)
        val_loss_list.append(val_loss)
        f1_list.append(f1)
        precision_list.append(precision)
        recall_list.append(recall)
        f1_class_lists.append(f1_class)
        precision_class_lists.append(precision_class)  # Append class-wise precision
        recall_class_lists.append(recall_class)  # Append class-wise recall

        # Save the model after each epoch if validation loss improves
        if val_loss < best_val_loss:
            torch.save(global_model.state_dict(), best_model_path)
            best_val_loss = val_loss

        # Print epoch statistics
        print(f'Epoch {epoch+1}/{num_epochs}, Training Loss: {epoch_train_loss:.4f}, Validation Loss: {val_loss:.4f}, Training Accuracy: {epoch_train_accuracy:.2f}%, Validation Accuracy: {val_accuracy:.2f}%, F1 Score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}')
        for i in range(len(f1_class)):
            print(f'Class {i}: F1 Score: {f1_class[i]:.4f}, Precision: {precision_class[i]:.4f}, Recall: {recall_class[i]:.4f}')



    result_df = pd.DataFrame({
        'Epoch': epoch_list,
        'Training Accuracy': train_accuracy_list,
        'Validation Accuracy': val_accuracy_list,
        'Training Loss': train_loss_list,
        'Validation Loss': val_loss_list,
        'F1 Score': f1_list,
        'Precision': precision_list,
        'Recall': recall_list,
    })

    # Add columns for class-wise metrics
    for i in range(len(f1_class_lists[0])):
        result_df[f'Class {i} F1 Score'] = [f1_class[i] for f1_class in f1_class_lists]
        result_df[f'Class {i} Precision'] = [precision_class[i] for precision_class in precision_class_lists]
        result_df[f'Class {i} Recall'] = [recall_class[i] for recall_class in recall_class_lists]

    # Save result_df to a CSV file
    result_df.to_csv('/content/drive/MyDrive/Colab/IoTDevice/Federated_IID_Best/FocalLoss/result_df_iid_focalloss.csv', index=False)


In [61]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

class FocalLoss(nn.Module):
    def __init__(self, gamma=0, alpha=None, size_average=True):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        if isinstance(alpha, (float, int)):
            self.alpha = torch.Tensor([alpha, 1 - alpha])
        if isinstance(alpha, list):
            self.alpha = torch.Tensor(alpha)
        self.size_average = size_average

    def forward(self, input, target):
        if input.dim() > 2:
            input = input.view(input.size(0), input.size(1), -1)  # N,C,H,W => N,C,H*W
            input = input.transpose(1, 2)    # N,C,H*W => N,H*W,C
            input = input.contiguous().view(-1, input.size(2))   # N,H*W,C => N*H*W,C
        target = target.view(-1, 1)

        logpt = F.log_softmax(input, dim=1)
        logpt = logpt.gather(1, target)
        logpt = logpt.view(-1)
        pt = Variable(logpt.data.exp())

        if self.alpha is not None:
            if self.alpha.type() != input.data.type():
                self.alpha = self.alpha.type_as(input.data)
            at = self.alpha.gather(0, target.data.view(-1))
            logpt = logpt * Variable(at)

        loss = -1 * (1 - pt) ** self.gamma * logpt
        if self.size_average:
            return loss.mean()
        else:
            return loss.sum()


In [62]:
alpha = [0.8] * 18  # Initialize alpha with all values set to 0.8
alpha[5] = alpha[11] = alpha[16] = 1

In [63]:
num_epochs = 200
focal_loss = FocalLoss(gamma=2, alpha=alpha)  # Example values for gamma and alpha
criterion = focal_loss  # Set the criterion for training
optimizer = torch.optim.Adam(global_model.parameters(), lr=0.001)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_federated(global_model, client_models, train_loaders, val_loaders, num_epochs, criterion, optimizer, device)

[0.008552371291715502, 0.005396668630510104, 0.00401297353653218, 0.003312312726412758, 0.002813926887910731]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/200, Training Loss: 0.0048, Validation Loss: 0.0019, Training Accuracy: 72.69%, Validation Accuracy: 87.28%, F1 Score: 0.6605, Precision: 0.6927, Recall: 0.6566
Class 0: F1 Score: 0.8031, Precision: 0.8110, Recall: 0.7953
Class 1: F1 Score: 0.8282, Precision: 0.8227, Recall: 0.8338
Class 2: F1 Score: 0.8742, Precision: 0.8473, Recall: 0.9029
Class 3: F1 Score: 0.9801, Precision: 0.9850, Recall: 0.9753
Class 4: F1 Score: 0.8516, Precision: 0.7580, Recall: 0.9716
Class 5: F1 Score: 0.3066, Precision: 0.3818, Recall: 0.2561
Class 6: F1 Score: 0.4332, Precision: 0.6493, Recall: 0.3250
Class 7: F1 Score: 0.9057, Precision: 0.8770, Recall: 0.9363
Class 8: F1 Score: 0.8742, Precision: 0.8400, Recall: 0.9112
Class 9: F1 Score: 0.8634, Precision: 0.8734, Recall: 0.8536
Class 10: F1 Score: 0.6402, Precision: 0.6294, Recall: 0.6514
Class 11: F1 Score: 0.0000, Precision: 0.0000, Recall: 0.0000
Class 12: F1 Score: 0.9064, Precision: 0.8770, Recall: 0.9377
Class 13: F1 Score: 0.1667, Precisi

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 2/200, Training Loss: 0.0020, Validation Loss: 0.0013, Training Accuracy: 85.88%, Validation Accuracy: 90.83%, F1 Score: 0.7415, Precision: 0.7820, Recall: 0.7243
Class 0: F1 Score: 0.8473, Precision: 0.7868, Recall: 0.9180
Class 1: F1 Score: 0.8720, Precision: 0.9406, Recall: 0.8127
Class 2: F1 Score: 0.9074, Precision: 0.9147, Recall: 0.9003
Class 3: F1 Score: 0.9820, Precision: 0.9833, Recall: 0.9808
Class 4: F1 Score: 0.9269, Precision: 0.8844, Recall: 0.9737
Class 5: F1 Score: 0.3125, Precision: 0.4348, Recall: 0.2439
Class 6: F1 Score: 0.6526, Precision: 0.5981, Recall: 0.7180
Class 7: F1 Score: 0.9320, Precision: 0.9475, Recall: 0.9169
Class 8: F1 Score: 0.8951, Precision: 0.8641, Recall: 0.9283
Class 9: F1 Score: 0.9086, Precision: 0.9018, Recall: 0.9155
Class 10: F1 Score: 0.8237, Precision: 0.8330, Recall: 0.8147
Class 11: F1 Score: 0.0000, Precision: 0.0000, Recall: 0.0000
Class 12: F1 Score: 0.9525, Precision: 0.9679, Recall: 0.9377
Class 13: F1 Score: 0.4875, Precisi

In [82]:
model = TransformerClassifier(input_dim, hidden_dim, num_layers, num_classes)
 # Instantiate your model class
model.load_state_dict(torch.load('/content/drive/MyDrive/Colab/IoTDevice/Federated_IID_Best/FocalLoss/best_model_focalloss.pth')) # Load the trained weights

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


<All keys matched successfully>

In [83]:
# TESTING THE MODEL AFTER TRAINING

criterion = nn.CrossEntropyLoss()
# Initialize lists to store results
all_predicted = []
all_labels = []

# Testing loop
model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    for inputs, labels in test_loaders[1]:
        inputs = torch.unsqueeze(inputs, 0)
        labels = torch.squeeze(labels, dim=1)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        all_predicted.extend(predicted.tolist())
        all_labels.extend(labels.tolist())

# Calculate test metrics
test_f1 = f1_score(all_labels, all_predicted, average='macro')
test_precision = precision_score(all_labels, all_predicted, average='macro')
test_recall = recall_score(all_labels, all_predicted, average='macro')

print(f"Test F1 Score: {test_f1:.4f}")
print(f"Test Precision: {test_precision:.4f}")
print(f"Test Recall: {test_recall:.4f}")

test_f1_class = f1_score(all_labels, all_predicted, average=None)
test_precision_class = precision_score(all_labels, all_predicted, average=None)
test_recall_class = recall_score(all_labels, all_predicted, average=None)

# Create DataFrame for results
result_df = pd.DataFrame({
    'Class': range(len(test_f1_class)),
    'F1 Score': test_f1_class,
    'Precision': test_precision_class,
    'Recall': test_recall_class
})

# Save result_df to a CSV file
result_df.to_csv('/content/drive/MyDrive/Colab/IoTDevice/Federated_IID_Best/FocalLoss/model_result_focalloss.csv', index=False)

Test F1 Score: 0.9532
Test Precision: 0.9609
Test Recall: 0.9498


In [84]:
# Test after training

from sklearn.metrics import confusion_matrix

# Initialize lists to store results
all_predicted = []
all_labels = []

# Testing loop
model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    for val_loader in test_loaders:
      for inputs, labels in val_loader:
        inputs = torch.unsqueeze(inputs, 0)
        labels = torch.squeeze(labels, dim=1)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        all_predicted.extend(predicted.tolist())
        all_labels.extend(labels.tolist())

# Calculate test metrics
test_f1 = f1_score(all_labels, all_predicted, average='macro')
test_precision = precision_score(all_labels, all_predicted, average='macro')
test_recall = recall_score(all_labels, all_predicted, average='macro')

# Calculate confusion matrix
conf_matrix = confusion_matrix(all_labels, all_predicted)

print(f"Test F1 Score: {test_f1:.4f}")
print(f"Test Precision: {test_precision:.4f}")
print(f"Test Recall: {test_recall:.4f}")
print("Confusion Matrix:")


# Convert confusion matrix to DataFrame
conf_matrix_df = pd.DataFrame(conf_matrix)

# Save DataFrame to CSV
conf_matrix_df.to_csv('/content/drive/MyDrive/Colab/IoTDevice/Federated_IID_Best/FocalLoss/confusion_matrix_focalloss.csv', index=False)

print("Confusion Matrix saved as confusion_matrix.csv")


Test F1 Score: 0.9532
Test Precision: 0.9609
Test Recall: 0.9498
Confusion Matrix:
Confusion Matrix saved as confusion_matrix.csv
